In [1]:
import sys
import cmath
import math
import os

if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *

from libra_py import units
from libra_py import data_conv
from libra_py import QE_methods

import libra_py.workflows.librax.defaults as defaults
import libra_py.workflows.librax.main as main

import libra_py.workflows.librax.x_to_libra_qe as x2liQE
import libra_py.workflows.librax.create_input_qe as create_input_qe
import libra_py.workflows.librax.extract_qe as extract_qe

import matplotlib.pyplot as plt   # plots
#import numpy as np
#from matplotlib.mlab import griddata

%matplotlib inline 

plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}

colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

clrs_index = ["11", "21", "31", "41", "12", "22", "32", "13","23", "14", "24"]

/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  if __name__ == '__main__':
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  if __name__ == '__main__':
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<in

In [3]:
params = {}

defaults.set_defaults(params, "QE")

for it in params.keys():
    print it, params[it]

Setting default parameters for Libra- QE  calculations
print_aux_results 1
tsh_method 1
sh_pop_ex_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/sh_pop_ex
do_rescaling 1
mu_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/mu
mo_ham /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/mo_ham/
traj_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/md
ent_file 
hop_probs /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/hop_probs/
sh_pop_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/sh_pop
sd_ham /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/sd_ham/
print_sd_ham 0
print_coherences 0
therm None
QM_fraction 1.0
res /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Exam

In [4]:
params["nproc"] = 1        # the number of processors
params["dt_nucl"]=20.0     # time step for nuclear dynamics  ex) 20 a.u. = 0.5 fsec
params["Nsnaps"]=5         # the number of MD rounds
params["Nsteps"]=2         # the number of MD steps per snap
params["Ncool"] = -1
params["nspin"] = 2
params["electronic_smearing"] = 0.01 # Electronic smearing used in Fermi population calculation
params["nconfig"] = 1
params["el_mts"] = 1
params["num_SH_traj"] = 1
params["scf_itr"] = 10     # Number of SCF steps in each fractional occupation update
params["max_iteration"] = 30 # Maximum number of fractional Fermi cycle
params["non-orth"] = 1     # = 1 when MOs are non-orthogonal, = 0 when calculated in orthogonal MO basis
params["print_S_mat"] = 0  # 1 if S-matrix printing required, 0 if not required
params["smat_inc"] = 0     # 1 Including overlap matrix (S), 0 when overlap matrix (S) not included in el propagation

params["MD_type"] = 1      # 1 NVT ensamble, 0 NVE ensamble

# Thermostat parameters
params["therm"] = Thermostat({"thermostat_type":"Nose-Hoover","nu_therm":0.001,"Temperature":300.0,"NHC_size":3})
params["Temperature"] = params["therm"].Temperature
params["sigma_pos"] = 0.01  #Displace atomic position randomly
params["Nstart"] = 0

params["excitations"] = [ excitation(0,1,0,1), excitation(0,1,1,1) ] 
params["excitations_init"] = [1]
params["HOMO"] = 0
params["min_shift"] = 0
params["max_shift"] = 1 
for i in range(0,len(params["excitations"])):
    params["qe_inp0%i" %i] = "x%i.scf.in" %i    # initial input file
    params["qe_inp%i" %i] = "x%i.scf_wrk.in" %i # working input file 
    params["qe_out%i" %i] = "x%i.scf.out" %i    # output file


for it in params.keys():
    print it, params[it]

print_aux_results 1
tsh_method 1
sh_pop_ex_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/sh_pop_ex
nspin 2
do_rescaling 1
dt_nucl 20.0
mu_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/mu
mo_ham /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/mo_ham/
excitations_init [1]
traj_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/md
ent_file 
hop_probs /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/hop_probs/
sh_pop_file_prefix /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/res/sh_pop
sd_ham /mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/sd_ham/
print_sd_ham 0
excitations [<liblibra_core.excitation object at 0x7fe811351ba8>, <liblibra_core.excitation object at 0x7fe811351c58>]
print_coherence

Run the QE calculations for initial input files: x0.scf.in  and x1.scf.in

In [6]:
nstates = len(params["excitations"])
for ex_st in xrange(nstates):
    print create_input_qe.read_qe_inp_templ("x%i.scf.in" % (ex_st) ) 

&CONTROL

  calculation = 'scf',

  pseudo_dir = '/mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example13_libraX/PP/',

  outdir = './',

  prefix = 'x0',

  tprnfor = .true.

  disk_io = 'low',

  wf_collect = .true.,

/



&SYSTEM

  ibrav = 0,

  celldm(1) = 1.89 ,

  nat = 6,

  ntyp = 2,

  nspin = 2,

  nbnd = 16,

  ecutwfc = 30,

  tot_charge = 0.0,

  starting_magnetization(1) = 0.01

  occupations = 'from_input',

  smearing = 'gaussian',

  degauss = 0.005,

  nosym = .true.,

/



&ELECTRONS

  electron_maxstep = 20,

  conv_thr = 1.D-5,

  mixing_beta = 0.45,

/



&IONS

  ion_dynamics = 'verlet',

  ion_temperature = 'andersen',

  tempw = 300.00 ,

  nraise = 1,

/





ATOMIC_SPECIES

 C  12.01  C.pbe-n-rrkjus_psl.1.0.0.UPF

 H  1.008  H.pbe-rrkjus_psl.1.0.0.UPF



K_POINTS gamma

                               

CELL_PARAMETERS

     8.0000000    0.0000000    0.0000000

     0.0000000    8.0000000    0.0000000

     0.0000000    0.0000000    8.000

In [ ]:
########### Now start actual calculations ###########################

qe_run = { "EXE"       :"/mnt/c/cygwin/home/Alexey-user/Soft/espresso/bin/pw.x", 
           "EXE_EXPORT":"/mnt/c/cygwin/home/Alexey-user/Soft/espresso/bin/pw_export.x",
           "BATCH_SYSTEM":None,
           "NP": 1
         }


nstates = len(params["excitations"])
for ex_st in xrange(nstates):
    os.system("cp x%i.scf.in x%i.scf_wrk.in" % (ex_st, ex_st))
    x2liQE.exe_espresso(ex_st, qe_run)

Check if the convergence was achieved for each calculation

In [4]:
print x2liQE.check_convergence("x0.scf.out")
print x2liQE.check_convergence("x1.scf.out")

True
False


Extract the essential information from the completed run x0.scf.out

In [5]:
info = QE_methods.read_qe_schema("x0.save/data-file-schema.xml")
print info

print "Coordinates"
for at in xrange(info["nat"]):
    print at, info["coords"].get(3*at, 0), info["coords"].get(3*at+1, 0), info["coords"].get(3*at+2, 0)

print "Forces"
for at in xrange(info["nat"]):
    print at, info["forces"].get(3*at, 0), info["forces"].get(3*at+1, 0), info["forces"].get(3*at+2, 0)

{'atom_labels': ['C', 'C', 'H', 'H', 'H', 'H'], 'nbnd': 32, 'forces': <liblibra_core.MATRIX object at 0x7ff2a03a5520>, 'conv': 6, 'etot': -14.75620293061859, 'efermi': -0.2311576402336118, 'coords': <liblibra_core.MATRIX object at 0x7ff2a03a5050>, 'alat': 15.12, 'nat': 6, 'nelec': 12}
Coordinates
0 -9.53981850969 4.38031311732 0.03386101698
1 -7.02089431344 4.36381756587 -0.0113665167
2 -10.6521664712 2.92591307106 -0.91981436652
3 -10.5978628136 5.8489221609 1.0266796386
4 -5.96214893106 2.89553749569 -1.00394712684
5 -5.90880186099 5.81814849105 0.94238165637
Forces
0 0.000765965171835 -0.00150633883548 0.000714668698357
1 -0.00169663823829 -0.000774222903868 -0.000289596051506
2 -0.00023750100707 -0.000183484426975 -0.000616272533866
3 -0.0001825855177 0.00130450819965 0.000462929400444
4 0.00065691309728 -0.000196193136767 -0.000701771690123
5 0.000693846493945 0.00135573110344 0.000430042176695


So, there are 12 electrons in our system (C2H4 - 4 * 2 + 4 = 12), so there will be 6 occupied orbitals

Our excitations involve HOMO -> LUMO transition, so we'll need orbitals 6 and 7

In [6]:
active_space = main.construct_active_space(info["nelec"], params["excitations"] )

print active_space

[6, 7]


In [7]:
nex = len(params["excitations"])

for ex in range(nex):
    print "Excitation %i " % (ex)

    occ, occ_alp, occ_bet = create_input_qe.excitation_to_qe_occ(info["nbnd"]/2, info["nelec"], params["excitations"][ex])

    print occ
    print occ_alp
    print occ_bet
    
    print create_input_qe.print_occupations(occ)
    print create_input_qe.print_occupations(occ_alp)
    print create_input_qe.print_occupations(occ_bet)
    
    nspin = 2
    scf_iter = 1
    restart_flag = 0 #  1 - change scf_iter, 2 - change scf_iter + restart from the previous pot and wfc
    
    create_input_qe.write_qe_input_first("x%i.scf_wrk.in" % (ex), "x%i.scf_wrk-test.in" % (ex),
                                         occ, occ_alp, occ_bet, nspin, scf_iter, restart_flag)

Excitation 0 
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  2.00000000   2.00000000   2.00000000   2.00000000   2.00000000   2.00000000   0.00000000   0.00000000   0.00000000   0.00000000 
  0.00000000   0.00000000   0.00000000   0.00000000   0.00000000   0.00000000 

  1.00000000   1.00000000   1.00000000   1.00000000   1.00000000   1.00000000   0.00000000   0.00000000   0.00000000   0.00000000 
  0.00000000   0.00000000   0.00000000   0.00000000   0.00000000   0.00000000 

  1.00000000   1.00000000   1.00000000   1.00000000   1.00000000   1.00000000   0.00000000   0.00000000   0.00000000   0.00000000 
  0.00000000   0.00000000   0.00000000   0.00000000   0.00000000   0.00000000 

Excitation 1 
[2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0,

In [8]:
nspin = 2
ex_st = 0

info, MO_a, MO_b = extract_qe.qe_extract(ex_st, active_space, nspin)

print info

print "Coordinates"
for at in xrange(info["nat"]):
    print at, info["coords"].get(3*at, 0), info["coords"].get(3*at+1, 0), info["coords"].get(3*at+2, 0)

print "Forces"
for at in xrange(info["nat"]):
    print at, info["forces"].get(3*at, 0), info["forces"].get(3*at+1, 0), info["forces"].get(3*at+2, 0)
    
print MO_a.num_of_rows, MO_a.num_of_cols

path= Kpoint.1
path= Kpoint.1
path= Kpoint.2
path= Kpoint.2
{'atom_labels': ['C', 'C', 'H', 'H', 'H', 'H'], 'nbnd': 32, 'forces': <liblibra_core.MATRIX object at 0x7ff2a03a50a8>, 'conv': 6, 'etot': -14.75620293061859, 'efermi': -0.2311576402336118, 'coords': <liblibra_core.MATRIX object at 0x7ff2a03a5788>, 'alat': 15.12, 'nat': 6, 'nelec': 12}
Coordinates
0 -9.53981850969 4.38031311732 0.03386101698
1 -7.02089431344 4.36381756587 -0.0113665167
2 -10.6521664712 2.92591307106 -0.91981436652
3 -10.5978628136 5.8489221609 1.0266796386
4 -5.96214893106 2.89553749569 -1.00394712684
5 -5.90880186099 5.81814849105 0.94238165637
Forces
0 0.000765965171835 -0.00150633883548 0.000714668698357
1 -0.00169663823829 -0.000774222903868 -0.000289596051506
2 -0.00023750100707 -0.000183484426975 -0.000616272533866
3 -0.0001825855177 0.00130450819965 0.000462929400444
4 0.00065691309728 -0.000196193136767 -0.000701771690123
5 0.000693846493945 0.00135573110344 0.000430042176695
9627 2


In [9]:
nbnd = info["nbnd"]
info1, all_e = QE_methods.read_qe_index("x0.export/index.xml", range(1, nbnd/2 + 1), 0)

path= Root


In [10]:
print info1, all_e
print all_e[0].num_of_rows, all_e[0].num_of_cols

bnds_alp, bnds_bet = [], []
for i in xrange(nbnd/2):
    print i, all_e[0].get(i,i).real, all_e[1].get(i,i).real
    bnds_alp.append(all_e[0].get(i,i).real)
    bnds_bet.append(all_e[1].get(i,i).real)

{'a1': <liblibra_core.VECTOR object at 0x7ff2a03a57e0>, 'nbnd': 16, 'nspin': 2, 'k': [<liblibra_core.VECTOR object at 0x7ff2a03a5af8>, <liblibra_core.VECTOR object at 0x7ff2a03a5b50>], 'tpiba': 3.32443667046539, 'efermi': -0.2311576402336118, 'a3': <liblibra_core.VECTOR object at 0x7ff2a03a5998>, 'nk': 2, 'alat': 1.89, 'b1': <liblibra_core.VECTOR object at 0x7ff2a03a59f0>, 'b2': <liblibra_core.VECTOR object at 0x7ff2a03a5a48>, 'b3': <liblibra_core.VECTOR object at 0x7ff2a03a5aa0>, 'weights': [1.0, 1.0], 'a2': <liblibra_core.VECTOR object at 0x7ff2a03a5940>, 'omega': 3456.649728, 'tpiba2': 11.05187917593501} [<liblibra_core.CMATRIX object at 0x7ff2a03a5ba8>, <liblibra_core.CMATRIX object at 0x7ff2a03a5c00>]
16 16
0 -0.674505444989 -0.674483646413
1 -0.505399411815 -0.505385772048
2 -0.403150333432 -0.403142690558
3 -0.357688374905 -0.357677872333
4 -0.294638912884 -0.29464606227
5 -0.231177126918 -0.231157640234
6 -0.0445974209132 -0.0446048218575
7 -0.0223830383064 -0.0223654371343
8 0

In [11]:
nalp = info["nelec"]/2
nbet = info["nelec"] - nalp
kT = 0.001

Ef_a = fermi_energy(bnds_alp, nalp, 1, kT, 1e-10)
Ef_b = fermi_energy(bnds_bet, nalp, 1, kT, 1e-10)

print "Ef_a = ", Ef_a
print "Ef_b = ", Ef_b

for i in xrange(nbnd/2):
    print i, fermi_population(bnds_alp[i], Ef_a, 1, kT), fermi_population(bnds_bet[i], Ef_b, 1, kT)


Ef_a =  -0.0799479270646
Ef_b =  -0.0799553280874
0 1.0 1.0
1 1.0 1.0
2 1.0 1.0
3 1.0 1.0
4 1.0 1.0
5 1.0 1.0
6 4.44089235243e-16 4.44089200376e-16
7 0.0 0.0
8 0.0 0.0
9 0.0 0.0
10 0.0 0.0
11 0.0 0.0
12 0.0 0.0
13 0.0 0.0
14 0.0 0.0
15 0.0 0.0


In [14]:
nspin = 2

pop = extract_qe.excited_populations(bnds_alp, info["nelec"], nspin, kT, 0)
print pop

pop1 = extract_qe.excited_populations(bnds_alp, info["nelec"], nspin, kT, 1)
print pop1

pop2 = extract_qe.excited_populations(bnds_alp, info["nelec"], nspin, kT, 2)
print pop2

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.44089235242551e-16, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 0.9999999999999833, 1.6653345369377348e-14, 0.999984996188551, 1.5003811005201348e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 0.9999999999999833, 1.6653345369377348e-14, 1.5003811448588422e-05, 0.9999849961680165, 1.8200021335597112e-11, 2.6427261145793925e-12, 1.2301409495984832e-13, 1.336194690192103e-14, 2.0600649336716e-18, 0.0, 0.0, 0.0]
